In [7]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/Colab Notebooks/Notebooks'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import numpy as np
import pandas as pd
import lightgbm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_auc_score

In [49]:
data = pd.read_csv(path + '/train-2.csv')

In [51]:
y = data['TARGET']
X = data.drop(columns = 'TARGET')
X = X.drop(columns = 'SK_ID_CURR')

In [52]:
submission = pd.read_csv(path + '/test-2.csv')
ids = submission['SK_ID_CURR'].values
submission.drop('SK_ID_CURR', inplace=True, axis=1)


X_test = submission.drop(columns = 'TARGET')
y_test = submission['TARGET']


In [53]:
categorical = list(X.dtypes[X.dtypes == "object"].index)
for c, col in enumerate(X.columns):
   if ('FLAG' in col):
       categorical.append(col)
numerical = list(set(X.columns) - set(categorical))


In [54]:
X_mean = X[numerical].mean()
X[numerical] = X[numerical].fillna(X_mean)
X[categorical] = X[categorical].fillna('-1')
#X, X_valid, y, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [65]:
train_data = lightgbm.Dataset(X, label=y, categorical_feature=categorical)
test_data = lightgbm.Dataset(X_test, label=y_test)

In [68]:
from lightgbm import LGBMClassifier
clf = LGBMClassifier()
clf.fit(X[numerical], y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [57]:
predictions = clf.predict_proba(X_test[numerical])[:, 1]

In [58]:

roc_auc_score(y_test, clf.predict_proba(X_test[numerical])[:, 1])

0.7001429724246099